<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/risk/frequencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pandas_datareader import DataReader as pdr

# -------------------
# annual data example
# -------------------

# read market return for famafrench
ffa = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[1]/100
mkta = ffa["Mkt-RF"] + ffa.RF
mkta = pd.DataFrame(mkta).reset_index()
mkta.columns = ['date', 'ret']
mkta.date = mkta.date.astype(str)

mkta

,date,ret
0,1927,0.3259
1,1928,0.3895
2,1929,-0.1479
3,1930,-0.2882
4,1931,-0.4404
...,...,...
90,2017,0.2231
91,2018,-0.0512
92,2019,0.3043
93,2020,0.2411


In [ ]:
# example stock return
ticker = 'CVX'
close = pdr(ticker, "yahoo", start=1970)["Adj Close"]

annual = close.resample('Y').last().pct_change().dropna().reset_index()
annual.columns = ['date', 'ret']
annual.date = annual.date.dt.to_period('Y').astype(str)

annual

,date,ret
0,1971,0.106522
1,1972,0.453690
2,1973,-0.090197
3,1974,-0.316027
4,1975,0.417095
5,1976,0.481889
6,1977,-0.051830
7,1978,0.205788
8,1979,0.202667
9,1980,0.764967


In [ ]:
tbla = pd.DataFrame(annual.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]).iloc[1:])

tbla

,ret
mean,0.137380
std,0.225215
min,-0.316027
10%,-0.159355
25%,-0.017183
50%,0.116747
75%,0.289680
90%,0.412047
max,0.764967
